# System of Linear Equations
---
**Name:** Jonh Alexis Buot <br>
**Date:** December 2023 <br>
**Course:** CS3101N <br>
**Task:** Assignment - System of Linear Equations

---

## Assignment 1

### Code for Backwards Substitution

In [43]:
import math
import numpy as np

def IsInRowEchelonForm(A):
# =============================================================================
#   Checks whether A is in Row Echelon Form. A is a 2D numpy array.
#   A matrix in row echelon form, must satisfy two things:
#   1. Any rows which are majority zeroes must be at the bottom.
#   2. In each non-zero row, the first nonzero entry (leading entry) is in a column to the left of the leading entry below it.
# =============================================================================
    first_nonzero_indices = np.argmax(A != 0, axis=1)

    # Strip zero rows that are found at the bottom of the matrix
    first_nonzero_indices = first_nonzero_indices[:len(first_nonzero_indices) - np.argmax(first_nonzero_indices[::-1] != 0)]
    diagonal_leading_ones = all(curr < next for curr, next in zip(first_nonzero_indices[:-1], first_nonzero_indices[1:]))
    
    return diagonal_leading_ones

def BackwardsSubstitute(A):
# =============================================================================
#   Performs Backward Substitution on matrix A, if matrix A is already in
#   Row Echelon Form. If the given matrix is inconsistent, it returns none.
#   If the given matrix has infinite solutions, it returns inf, else it returns
#   the solution vector.
# =============================================================================
    assert IsInRowEchelonForm(A), "Matrix A should be in row echelon form, or reduced row echelon form."

    # Strip zero rows if the variables are equal to the number of rows, this is to account for extra-rowed consistent matrices
    buffer_A = np.copy(A)
    while np.all(buffer_A[-1] == 0):
        buffer_A = np.delete(buffer_A, -1, axis=0)
    
    if np.all(buffer_A[-1, :-1] == 0) and buffer_A[-1, -1] != 0:
        return None
    elif buffer_A.shape[0] < buffer_A.shape[1] - 1:
        return math.inf

    flipped_matrix = buffer_A[::-1, ::-1]
    solution_vector, coefficient_matrix = flipped_matrix[:, 0], flipped_matrix[:, 1:]

    for i, row in enumerate(coefficient_matrix):
        # Let's say you're solving for x in 3x + 2y + 4z = b => x = (b - 2y + 4z) / 3
        # sum of other vars is the 2y + 4z part, 
        # 3 is the scaling part (row[i]), and 
        # b is the constant (solution_vector[i]) 
        sum_of_other_vars = sum(np.delete(solution_vector, i) * np.delete(row, i))
        solution_vector[i] = (solution_vector[i] - sum_of_other_vars) / row[i]

    return solution_vector[::-1]

A = np.array([[1, -1, -2, 5],
              [0,  1,  1, 3],
              [0,  0,  1, 2]])

solution_vector = BackwardsSubstitute(A)
print(solution_vector)

[10  1  2]


### Backwards Substitution Testing

Solve the following system of equations in `Python`, the solution should be $x_1 = 1$, $x_2 = 0$, $x_3 = 2$.
$$
\begin{align}
    x_1 - x_2 + x_3 &= 3 \\
    2 x_1 + x_2 + 8 x_3 &= 18 \\
    4 x_1 + 2 x_2 - 3 x_3 &= -2 \\
\end{align}
$$

In [44]:
def RowSwap(A,k,l):
# =============================================================================
#     A is a NumPy array.  RowSwap will return duplicate array with rows
#     k and l swapped.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def RowScale(A,k,scale):
# =============================================================================
#     A is a NumPy array.  RowScale will return duplicate array with the
#     entries of row k multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale
        
    return B

def RowAdd(A,k,l,scale):
# =============================================================================
#     A is a numpy array.  RowAdd will return duplicate array with row
#     l modifed.  The new values will be the old values of row l added to 
#     the values of row k, multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B

A = np.array([[1,-1,1,3],[2,1,8,18],[4,2,-3,-2]])

## Add -2 times row 0 to row 1
A1 = RowAdd(A, 0, 1, -2)
print("A1")
print(A1, "\n")

## Add -4 times row 0 to row 2
A2 = RowAdd(A1, 0, 2, -4)
print("A2")
print(A2, "\n")

## Add -2 times row 1 to row 2
A3 = RowAdd(A2, 1, 2, -2)
print("A3")
print(A3, "\n")

## Multiply row 1 by 1/3
A4 = RowScale(A3, 1, 1.0/3)
print("A4")
print(A4, "\n")

## Multiply row 2 by 1/19
A5 = RowScale(A4, 2, 1.0/-19.)
print("A5")
print(A5, "\n")

solution = BackwardsSubstitute(A5)
print("Solution:", solution, "\n")

print(f"x_1 = {solution[0]}", f"x_2 = {solution[1]}", f"x_3 = {solution[2]}", sep=", ")

A1
[[ 1. -1.  1.  3.]
 [ 0.  3.  6. 12.]
 [ 4.  2. -3. -2.]] 

A2
[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   6.  -7. -14.]] 

A3
[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   0. -19. -38.]] 

A4
[[  1.  -1.   1.   3.]
 [  0.   1.   2.   4.]
 [  0.   0. -19. -38.]] 

A5
[[ 1. -1.  1.  3.]
 [ 0.  1.  2.  4.]
 [-0. -0.  1.  2.]] 

Solution: [1. 0. 2.] 

x_1 = 1.0, x_2 = 0.0, x_3 = 2.0


# Assignment 2

**1. Provide reasons why each of the following equations is, or is not, a linear equation.**

- $\cos\theta x + 4y + z = \sqrt{3} \rightarrow \fbox{Not a linear equation.}$ <br>
**Solution:** $\cos\theta x + 4y + z = \sqrt{3}$ is not a linear equation, since it not follow the form $a_1 x_1 + a_2 x_2 + a_3 x_3 = b$, where $a_1, a_2, a_3, b$ are constants. This is because the coefficient, $\cos\theta$, of the first expression is not a constant.

- $\cos x 4y + z = \sqrt{3} \rightarrow \fbox{Not a linear equation.}$ <br>
**Solution:** $\cos x 4y + z = \sqrt{3}$ is not a linear equation, since it does not follow the form $a_1 x_1 + a_2 x_2 = b$, where $a_1, a_2, b$ are constants. This is because the coefficient, $4\cos x$, of the first expression is not a constant, assuming that $x_1 = y$.

- $x^{-1} + 7y + z = \sin(\frac{\pi}{2}) \rightarrow \fbox{Not a linear equation.}$ <br>
**Solution:** $x^{-1} + 7y + z = \sin(\frac{\pi}{2})$ is not a linear equation, since it does not follow the form $a_1 x_1 + a_2 x_2 + a_3 x_3 = b$, where $a_1, a_2, a_3, b$ are constants. This is because not all variables are not all raised to a degree of one.

- $x^1 + y^1 + z^1 = 0 \rightarrow \fbox{Linear equation.}$ <br>
**Solution** $x^1 + y^1 + z^1 = 0$ is of the form $a_1 x_1 + a_2 x_2 + a_3 x_3 = b$ so it is a linear equation

**2. Find the solutions to the linear equations.**

- $2x_1 + 3x_2 = 5$ <br>
**Solution** <br>
We need to transform the linear equation into an `augmented matrix`. Let $x_2 = s$. By substitution, we have $2x_1 + 3s = 5$. Solving for $x_1$ yields $x_1 = \frac{5 - 3s}{2}$. An `augmented matrix` can then be created of the form: <br>
$ 
\begin{align}
    \begin{pmatrix}
        1 & 0 &\bigm| & \frac{5 - 3s}{2} \\
        0 & 1 &\bigm| & s \\
    \end{pmatrix} 
\end{align}
$
We can have any arbitrary value of $s$ that will satisfy $\frac{5 - 3s}{2}$, this arbitrary value is the solution to $x_2$, while the result from the formula $\frac{5 - 3s}{2}$ is the solution to $x_1$.

- $4x_1 + 3x_2 + 2x_3 = 1$ <br>
**Solution** <br>
We need to transform the linear equation into an `augmented matrix`. Let $x_2 = s$ and $x_3 = t$. By substitution, we have $4x_1 + 3s + 2t = 1$. Solving for $x_1$ yields $x_1 = \frac{1 - 3s - 2t}{4}$. An `augmented matrix` can then be created of the form: <br>
$ 
\begin{align}
    \begin{pmatrix}
        1 & 0 & 0 &\bigm| & \frac{1 - 3s - 2t}{4} \\
        0 & 1 & 0 &\bigm| & s \\
        0 & 0 & 1 &\bigm| & t \\
    \end{pmatrix} 
\end{align}
$
We can have any arbitrary values of $s$ and $t$ that will satisfy $\frac{1 - 3s - 2t}{4}$, these arbitrary values are the solution to $x_2$ and $x_3$, respectively, while the result from the formula $\frac{1 - 3s - 2t}{4}$ is the solution to $x_1$.

- $3x - 6y = 0$ <br>
We need to transform the linear equation into an `augmented matrix`. Let $y = s$. By substitution, we have $3x - 6s = 0$. Solving for $x$ yields $x = 2s$. An `augmented matrix` can then be created of the form: <br>
$ 
\begin{align}
    \begin{pmatrix}
        1 & 0 &\bigm| & 2s \\
        0 & 1 &\bigm| & s \\
    \end{pmatrix} 
\end{align}
$
We can have any arbitrary value of $s$ that will satisfy $2s$, this arbitrary value is the solution to $y$, while the result from the formula $2s$ is the solution to $x$.

**3. Solve the following systems of linear equations**

For manual solutions, `augmented matrices` are reduced down to reduced row echelon forms, while for `python` solutions `augmented matrices` are reduced only to row echelon forms. This is to showcase the `BackwardsSubstitute` function.

a. 
$$
\begin{align}
    x - y = 0 \\
    2x + y = 3 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{pmatrix}
    1 & -1 &\bigm| & 0 \\
    2 &  1 &\bigm| & 3 \\
\end{pmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{pmatrix}
        1 & -1 &\bigm| & 0 \\
        2 &  1 &\bigm| & 3 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} - 2 \text{R}_{1}}
    \begin{pmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  3 &\bigm| & 3 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  3 &\bigm| & 3 \\
    \end{pmatrix}
    \xrightarrow{\frac{1}{3}\text{R}_{2}}
    \begin{pmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  1 &\bigm| & 1 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  1 &\bigm| & 1 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} + \text{R}_{2}}
    \begin{pmatrix}
        1 & 0 &\bigm| & 1 \\
        0 & 1 &\bigm| & 1 \\
    \end{pmatrix}
\end{align}
$$

$\therefore$ The solution to the systems of linear equations are $\boxed{x = 1 \text{ and } y = 1}$. <br>

**Pythonic Solution**

In [45]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, -1, 0],
              [2, 1 ,3]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, -2)
print("R2 - 2R1\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", sep=", ")

R2 - 2R1
 [[ 1. -1.  0.]
 [ 0.  3.  3.]] 

Solution: [1. 1.] 

x = 1.0, y = 1.0


b. 
$$
\begin{align}
    x + 5y = -1 \\
    -x + y = -5 \\
    2x + 4y = 4 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{pmatrix}
     1 &  5 &\bigm| & -1 \\
    -1 &  1 &\bigm| & -5 \\
     2 &  4 &\bigm| &  4 \\
\end{pmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{pmatrix}
        1 &  5 &\bigm| & -1 \\
        -1 &  1 &\bigm| & -5 \\
        2 &  4 &\bigm| &  4 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} + \text{R}_{1}}
    \begin{pmatrix}
        1 & 5 &\bigm| & -1 \\
        0 & 6 &\bigm| & -6 \\
        2 & 4 &\bigm| &  4 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 & 5 &\bigm| & -1 \\
        0 & 6 &\bigm| & -6 \\
        2 & 4 &\bigm| &  4 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} - 2 \text{R}_{1}}
    \begin{pmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 & -6 &\bigm| &  6 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 & -6 &\bigm| &  6 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{2}}
    \begin{pmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\frac{1}{6}\text{R}_{2}}
    \begin{pmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  1 &\bigm| & -1 \\
        0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  1 &\bigm| & -1 \\
        0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} - 5\text{R}_{2}}
    \begin{pmatrix}
        1 &  0 &\bigm| &  4 \\
        0 &  1 &\bigm| & -1 \\
        0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
\end{align}
$$

The solution is consistent since the last row is essentially $0x + 0y = 0$ which is true. <br>
$\therefore$ The solution to the systems of linear equations are $\boxed{x = 4 \text{ and } y = -1}$. <br>

**Pythonic Solution**

In [46]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 5, -1],
              [-1, 1, -5],
              [2, 4, 4]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, 1)
print("R2 + R1\n", A, "\n")

A = RowAdd(A, 0, 2, -2)
print("R3 - 2R1\n", A, "\n")

A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", sep=", ")

R2 + R1
 [[ 1.  5. -1.]
 [ 0.  6. -6.]
 [ 2.  4.  4.]] 

R3 - 2R1
 [[ 1.  5. -1.]
 [ 0.  6. -6.]
 [ 0. -6.  6.]] 

R3 + R2
 [[ 1.  5. -1.]
 [ 0.  6. -6.]
 [ 0.  0.  0.]] 

Solution: [ 4. -1.] 

x = 4.0, y = -1.0


c. 
$$
\begin{align}
    2x_1 + 3x_2 - x_3 = -1 \\
    x_1 + x_3 = 0 \\
    -x_1 + 2x_2 -2x_3 = 0 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{pmatrix}
     2 &  3 & -1 &\bigm| & -1 \\
     1 &  0 &  1 &\bigm| &  0 \\
    -1 &  2 & -2 &\bigm| &  0 \\
\end{pmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{pmatrix}
        2 &  3 & -1 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        -1 &  2 & -2 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{2}}
    \begin{pmatrix}
        2 &  3 & -1 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        2 &  3 & -1 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} - 2 \text{R}_{2}}
    \begin{pmatrix}
        0 &  3 & -3 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        0 &  3 & -3 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\frac{1}{3}\text{R}_{1}}
    \begin{pmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} - 2\text{R}_{1}}
    \begin{pmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} \leftrightarrow \text{R}_{2}}
    \begin{pmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{pmatrix}

    \\

    \begin{pmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} + \text{R}_{3}}
    \begin{pmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 &  0 &\bigm| &  \frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 &  0 &\bigm| &  \frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} - \text{R}_{3}}
    \begin{pmatrix}
        1 &  0 &  1 &\bigm| &  -\frac{2}{3} \\
        0 &  1 &  0 &\bigm| &  \frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{pmatrix}
\end{align}
$$

$\therefore$ The solution to the systems of linear equations are $\boxed{x_1 = -\frac{2}{3} \text{, } x_2 = \frac{1}{3} \text{, and } x_3 = \frac{2}{3}}$. <br>

**Pythonic Solution**

In [47]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[2, 3, -1, -1],
              [1, 0, 1, 0],
              [-1, 2, -2, 0]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

A = RowAdd(A, 1, 0, -2)
print("R1 - 2R2\n", A, "\n")

A = RowAdd(A, 0, 2, -2 / 3)
print("R3 - (2/3)R1\n", A, "\n")

A = RowSwap(A, 0, 1)
print("R1 <-> R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x_1 = {solution[0]}", f"x_2 = {solution[1]}", f"x_3 = {solution[2]}", sep=", ")

R3 + R2
 [[ 2.  3. -1. -1.]
 [ 1.  0.  1.  0.]
 [ 0.  2. -1.  0.]] 

R1 - 2R2
 [[ 0.  3. -3. -1.]
 [ 1.  0.  1.  0.]
 [ 0.  2. -1.  0.]] 

R3 - (2/3)R1
 [[ 0.          3.         -3.         -1.        ]
 [ 1.          0.          1.          0.        ]
 [ 0.          0.          1.          0.66666667]] 

R1 <-> R2
 [[ 1.          0.          1.          0.        ]
 [ 0.          3.         -3.         -1.        ]
 [ 0.          0.          1.          0.66666667]] 

Solution: [-0.66666667  0.33333333  0.66666667] 

x_1 = -0.6666666666666666, x_2 = 0.3333333333333333, x_3 = 0.6666666666666666


**4. Provide reasons why the given matrices are either a row echelon, a reduced row echelon, or neither both.**

a.
$$
\begin{align}
    \begin{pmatrix}
        1 & 0 & 1 \\
        0 & 0 & 3 \\
        0 & 1 & 0 \\
    \end{pmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since the leading non-zero entry of row 2 is to the right of the leading non-zero entry of the row below it, row 3.

b.
$$
\begin{align}
    \begin{pmatrix}
        7 & 0 &  1 & 0 \\
        0 & 1 & -1 & 4 \\
        0 & 0 &  0 & 0 \\
    \end{pmatrix}
\end{align}
$$
This is in $\fbox{row echelon form}$, since all zero rows are located at the bottom of the matrix, and each leading non-zero entry of each row is to the left of the leading non-zero entry of the row below it. However, it is not in reduced row echelon form, since the leading entry of the first row is not 1, and non-zero entries can be found outside of the leading entries.

c.
$$
\begin{align}
    \begin{pmatrix}
        0 & 1 & 3 & 0 \\
        0 & 0 & 0 & 1 \\
    \end{pmatrix}
\end{align}
$$
This is in $\fbox{row echelon form}$, since all zero rows are located at the bottom of the matrix, and each leading non-zero entry of each row is to the left of the leading non-zero entry of the row below it. However, it is not in reduced row echelon form even though its leading entries are all 1, since non-zero entries can be found outside of the leading entries.

d.
$$
\begin{align}
    \begin{pmatrix}
        0 & 0 & 0 \\
        0 & 0 & 0 \\
        0 & 0 & 0 \\
    \end{pmatrix}
\end{align}
$$
This is in $\fbox{reduced row echelon form}$, since it vacuously satisfies all the necessary requirements to be one. All zero rows are at the bottom of the matrix, each leading non-zero entry are to the left of the leading non-zero entry of the row below it, all leading entries are 1, and no non-zero entries can be found outside of the leading entries.

e.
$$
\begin{align}
    \begin{pmatrix}
        1 & 0 & 3 & -4 & 0 \\
        0 & 0 & 0 &  0 & 0 \\
        0 & 1 & 5 &  0 & 1 \\
    \end{pmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since a zero row can be found in between non-zero rows, which breaks the rule that all zero rows are found at the bottom of the matrix.

f.
$$
\begin{align}
    \begin{pmatrix}
        0 & 0 & 1 \\
        0 & 1 & 0 \\
        1 & 0 & 0 \\
    \end{pmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since leading non-zero entries are found to the right of the leading non-zero entry of the row below it.

g.
$$
\begin{align}
    \begin{pmatrix}
        1 & 2 & 1 \\
        1 & 0 & 0 \\
        0 & 1 & 1 \\
        0 & 0 & 1 \\
    \end{pmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since not all leading non-zero entries are to the left of the leading non-zero entry of the row below it, specifically the ones in row 1 and row 2.

h.
$$
\begin{align}
    \begin{pmatrix}
        2 & 1 & 3 & 5 \\
        0 & 0 & 1 & -1 \\
        0 & 0 & 0 & 3 \\
        0 & 0 & 0 & 0 \\
    \end{pmatrix}
\end{align}
$$
This is in $\fbox{row echelon form}$, since all zero rows are found at the bottom of the matrix, and all leading non-zero entries are to the left of the leading non-zero entry of the row below it. However, it is not in row reduced echelon form, since not all leading non-zero entries equal to 1, nor entries outside of the leading entries are equal to 0.

**5. Solve the following systems of linear equations**

For manual solutions, `augmented matrices` are reduced down to reduced row echelon forms, while for `python` solutions `augmented matrices` are reduced only to row echelon forms. This is to showcase the `BackwardsSubstitute` function.


a. 
$$
\begin{align}
    x_1 + 2x_2 - 3x_3 = 9 \\
    2x_1 - x_2 - x_3 = 0 \\
    4x_1 - x_2 + x_3 = 4 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{pmatrix}
    1 &  2 & -3 &\bigm| & 9 \\
    2 & -1 & -1 &\bigm| & 0 \\
    4 & -1 &  1 &\bigm| & 4 \\
\end{pmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        2 & -1 & -1 &\bigm| & 0 \\
        4 & -1 &  1 &\bigm| & 4 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} - 2\text{R}_{2}}
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        2 & -1 & -1 &\bigm| & 0 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        2 & -1 & -1 &\bigm| & 0 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} - 2 \text{R}_{1}}
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 & -5 &  5 &\bigm| & -18 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 & -5 &  5 &\bigm| & -18 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} + 5 \text{R}_{3}}
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  0 &  20 &\bigm| & 2 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  0 &  20 &\bigm| & 2 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} \leftrightarrow \text{R}_{3}}
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  20 &\bigm| & 2 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  20 &\bigm| & 2 \\
    \end{pmatrix}
    \xrightarrow{\frac{1}{20}\text{R}_{3}}
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{pmatrix}

    \\
    
    \begin{pmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} - 2\text{R}_{2}}
    \begin{pmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} - 3\text{R}_{3}}
    \begin{pmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  0 &\bigm| & \frac{37}{10} \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{pmatrix}

    \\

    \begin{pmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  0 &\bigm| & \frac{37}{10} \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} - 9\text{R}_{3}}
    \begin{pmatrix}
        1 &  0 &  0 &\bigm| & \frac{19}{10} \\
        0 &  1 &  0 &\bigm| & \frac{37}{10} \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{pmatrix}
\end{align}
$$

$\therefore$ The solution to the systems of linear equations are $\boxed{x_1 = \frac{19}{10} \text{, } x_2 = \frac{37}{10} \text{, and } x_3 = \frac{1}{10}}$. <br>

**Pythonic Solution**

In [48]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 2, -3, 9],
              [2, -1, -1, 0],
              [4, -1, 1, 4]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 1, 2, -2)
print("R3 - 2R2\n", A, "\n")

A = RowAdd(A, 0, 1, -2)
print("R2 - 2R1\n", A, "\n")

A = RowAdd(A, 2, 1, 5)
print("R2 + 5R3\n", A, "\n")

A = RowSwap(A, 1, 2)
print("R2 <-> R3\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x_1 = {solution[0]:.1f}", f"x_2 = {solution[1]}", f"x_3 = {solution[2]}", sep=", ")

R3 - 2R2
 [[ 1.  2. -3.  9.]
 [ 2. -1. -1.  0.]
 [ 0.  1.  3.  4.]] 

R2 - 2R1
 [[  1.   2.  -3.   9.]
 [  0.  -5.   5. -18.]
 [  0.   1.   3.   4.]] 

R2 + 5R3
 [[ 1.  2. -3.  9.]
 [ 0.  0. 20.  2.]
 [ 0.  1.  3.  4.]] 

R2 <-> R3
 [[ 1.  2. -3.  9.]
 [ 0.  1.  3.  4.]
 [ 0.  0. 20.  2.]] 

Solution: [1.9 3.7 0.1] 

x_1 = 1.9, x_2 = 3.7, x_3 = 0.1


b. 
$$
\begin{align}
    x + 2y= -1 \\
    2x + y + z = 1 \\
    -x + y - z = -1
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{pmatrix}
     1 &  2 &  0 &\bigm| & -1 \\
     2 &  1 &  1 &\bigm| &  1 \\
    -1 &  1 & -1 &\bigm| & -1 \\
\end{pmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{pmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        2 &  1 &  1 &\bigm| &  1 \\
        -1 &  1 & -1 &\bigm| & -1 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{1}}
    \begin{pmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        2 &  1 &  1 &\bigm| &  1 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        2 &  1 &  1 &\bigm| &  1 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} - 2 \text{R}_{1}}
    \begin{pmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        0 & -3 &  1 &\bigm| &  3 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        0 & -3 &  1 &\bigm| &  3 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{2}}
    \begin{pmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        0 & -3 &  1 &\bigm| &  3 \\
        0 &  0 &  0 &\bigm| &  1 \\
    \end{pmatrix}
\end{align}
$$

Looking at the last row yields $0x + 0y + 0z = 0 \neq 1$ which means this system of linear equations is inconsistent.

$\therefore$ This system of linear equations has $\fbox{no solutions}$. <br>

**Pythonic Solution**

In [49]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 2, 0, -1],
              [2, 1, 1, 1],
              [-1, 1, -1, -1]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 2, 1)
print("R3 + R1\n", A, "\n")

A = RowAdd(A, 0, 1, -2)
print("R2 - 2R1\n", A, "\n")

A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R3 + R1
 [[ 1.  2.  0. -1.]
 [ 2.  1.  1.  1.]
 [ 0.  3. -1. -2.]] 

R2 - 2R1
 [[ 1.  2.  0. -1.]
 [ 0. -3.  1.  3.]
 [ 0.  3. -1. -2.]] 

R3 + R2
 [[ 1.  2.  0. -1.]
 [ 0. -3.  1.  3.]
 [ 0.  0.  0.  1.]] 

Solution: None 



c. 
$$
\begin{align}
    x_1 - 3x_2 - 2x_3 = 0 \\
    -x_1 + 2x_2 + x_3 = 0 \\
    2x_1 + 4x_2 + 6x_3 = 0 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{pmatrix}
     1 & -3 & -2 &\bigm| & 0 \\
    -1 &  2 &  1 &\bigm| &  0 \\
     2 &  4 &  6 &\bigm| &  0 \\
\end{pmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{pmatrix}
        1 & -3 & -2 &\bigm| & 0 \\
        -1 &  2 &  1 &\bigm| &  0 \\
        2 &  4 &  6 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} + \text{R}_{1}}
    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        2 &  4 &  6 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        2 &  4 &  6 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} - 2 \text{R}_{1}}
    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 & 10 & 10 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 & 10 & 10 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{3} + 10 \text{R}_{2}}
    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{-1 \text{R}_{2}}
    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 &  1 &  1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 &  1 &  1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{1} + 3 \text{R}_{2}}
    \begin{pmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 &  1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{pmatrix}
\end{align}
$$

Looking at the final row echelon form, this system of equations has infinitely many solutions. By extracting the system of equations from the `augmented matrix` we have $x + z = 0$ and $y + z = 0$. $z$ is the common variable, so it is the only thing we have control of, we can then solve everything in terms of $x$ and $y$ yielding $y = -z$ and $y = -z$.

$\therefore$ This system of linear equations has $\fbox{infinitely many solutions}$. By having any arbitrary value for $z$, we can calculate the value of $x$ and $y$ by $x = -z$ and $y = -z$, respectively. <br>

**Pythonic Solution**

In [50]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, -3, -2, 0],
              [-1, 2, 1, 0],
              [2, 4, 6, 0]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, 1)
print("R2 + R1\n", A, "\n")

A = RowAdd(A, 0, 2, -2)
print("R3 - 2R1\n", A, "\n")

A = RowAdd(A, 1, 2, 10)
print("R3 + 10R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R2 + R1
 [[ 1. -3. -2.  0.]
 [ 0. -1. -1.  0.]
 [ 2.  4.  6.  0.]] 

R3 - 2R1
 [[ 1. -3. -2.  0.]
 [ 0. -1. -1.  0.]
 [ 0. 10. 10.  0.]] 

R3 + 10R2
 [[ 1. -3. -2.  0.]
 [ 0. -1. -1.  0.]
 [ 0.  0.  0.  0.]] 

Solution: inf 




d. 
$$
\begin{align}
    w + x + 2y + z = 1 \\
    w - x - y + z = 0 \\
    x + y = 1 \\
    w + x + z = 2
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{pmatrix}
    1 &  1 &  2 & 1 &\bigm| &  1 \\
    1 & -1 & -1 & 1 &\bigm| &  0 \\
    0 &  1 &  1 & 0 &\bigm| &  1 \\
    1 &  1 &  0 & 1 &\bigm| &  2 \\
\end{pmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        1 & -1 & -1 & 1 &\bigm| &  0 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        1 &  1 &  0 & 1 &\bigm| &  2 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{4} - \text{R}_{1}}
    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        1 & -1 & -1 & 1 &\bigm| &  0 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        1 & -1 & -1 & 1 &\bigm| &  0 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} - \text{R}_{1}}
    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 & -2 & -3 & 0 &\bigm| & -1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    
    \\
    
    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 & -2 & -3 & 0 &\bigm| & -1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} + 2 \text{R}_{3}}
    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{2} \leftrightarrow \text{R}_{3}}
    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    
    \\

    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{pmatrix}
    \xrightarrow{\text{R}_{4} - 2 \text{R}_{3}}
    \begin{pmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  0 &  0 & 0 &\bigm| & -1 \\
    \end{pmatrix}
\end{align}
$$

Looking at the last row yields $0w + 0x + 0y + 0z = 0 \neq -1$ which means this system of linear equations is inconsistent.

$\therefore$ This system of linear equations has $\fbox{no solutions}$. <br>

**Pythonic Solution**

In [55]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 1, 2, 1, 1],
              [1, -1, -1, 1, 0],
              [0, 1, 1, 0, 1],
              [1, 1, 0, 1, 2]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 3, -1)
print("R4 - R1\n", A, "\n")

A = RowAdd(A, 0, 1, -1)
print("R2 - R1\n", A, "\n")

A = RowAdd(A, 2, 1, 2)
print("R2 + 2R3\n", A, "\n")

A = RowSwap(A, 1, 2)
print("R2 <-> R3\n", A, "\n")

A = RowAdd(A, 2, 3, -2)
print("R4 - 2R3\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R4 - R1
 [[ 1.  1.  2.  1.  1.]
 [ 1. -1. -1.  1.  0.]
 [ 0.  1.  1.  0.  1.]
 [ 0.  0. -2.  0.  1.]] 

R2 - R1
 [[ 1.  1.  2.  1.  1.]
 [ 0. -2. -3.  0. -1.]
 [ 0.  1.  1.  0.  1.]
 [ 0.  0. -2.  0.  1.]] 

R2 + 2R3
 [[ 1.  1.  2.  1.  1.]
 [ 0.  0. -1.  0.  1.]
 [ 0.  1.  1.  0.  1.]
 [ 0.  0. -2.  0.  1.]] 

R2 <-> R3
 [[ 1.  1.  2.  1.  1.]
 [ 0.  1.  1.  0.  1.]
 [ 0.  0. -1.  0.  1.]
 [ 0.  0. -2.  0.  1.]] 

R4 - 2R3
 [[ 1.  1.  2.  1.  1.]
 [ 0.  1.  1.  0.  1.]
 [ 0.  0. -1.  0.  1.]
 [ 0.  0.  0.  0. -1.]] 

Solution: None 



**6. Determine if the vectors are linear combinations**